<a href="https://colab.research.google.com/github/nik280/COVID-CASES/blob/main/Major_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np #linear algebra
import pandas as pd #data preprocessing
import os
for dirname, _, filenames in os.walk('/content/covid19'):
     for filename in filenames:
       print(os.path.join(dirname, filename))

STATEWISE COVID TASTING


In [ ]:
Covid_Testing = pd.read_csv('/content/covid19/StatewiseTestingDetails.csv')
Covid_Testing.head()

In [ ]:
Covid_Testing['State'].unique()

In [ ]:
Covid_Testing.shape


In [ ]:
Covid_Testing.describe()

In [ ]:
print(Covid_Testing['Date'].min())
print(Covid_Testing['Date'].max())

INDIA COVID CASES

In [ ]:
Covid_Summary = pd.read_csv('/content/covid19/covid_19_india.csv')
Covid_Summary.head()

In [ ]:
Covid_Summary.describe()

In [ ]:
Covid_Summary.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')



In [ ]:
Covid_Summary.isnull().sum()

In [ ]:
Covid_Summary.info()

In [ ]:
Covid_Summary = Covid_Summary.drop(['Sno','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1)
Covid_Summary.head()

In [ ]:
Covid_Summary['Active'] = Covid_Summary['Confirmed'] - Covid_Summary['Cured'] - Covid_Summary['Deaths']
Covid_Summary.tail()

In [ ]:
Covid_Summary['Date'] = pd.to_datetime(Covid_Summary['Date'])
Covid_Summary.head()

In [ ]:
Covid_Summary.info()

In [ ]:
india_cases = Covid_Summary[Covid_Summary['Date'] == Covid_Summary['Date'].max()].copy().fillna(0)
india_cases.index = india_cases["State/UnionTerritory"]
india_cases = india_cases.drop(['State/UnionTerritory', 'Time','Date'], axis=1)

In [ ]:
india_cases.head()

In [ ]:
df = pd.DataFrame(pd.to_numeric(india_cases.sum())).transpose()
df.style.background_gradient(cmap='BuGn',axis=1)

In [ ]:
Trend = Covid_Summary.groupby(['Date'])['Confirmed', 'Deaths','Cured',].sum().reset_index()
Trend.head()

In [ ]:
fig = go.Figure(go.Bar(x= Trend.Date, y= Trend.Cured, name='Recovered'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Deaths, name='Deaths'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Confirmed, name='Confirmed'))

fig.update_layout(barmode='stack',legend_orientation="h",legend=dict(x= 0.3, y=1.1),
                 paper_bgcolor='white',
                 plot_bgcolor = "white",)
fig.show()

In [ ]:
import plotly.express as px

In [ ]:
def horizontal_bar_chart(df, x, y, title, x_label, y_label, color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, 
                 labels={x.name: x_label,
                         y.name: y_label}, color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [ ]:
No_death_states = india_cases.sort_values('Deaths',ascending = False)[:29]
horizontal_bar_chart(No_death_states, No_death_states.Deaths, No_death_states.index,
                     ' States with most deaths','Number of deaths(In Thousands)','State Name','Red')


In [ ]:
Confirmed_states_Cases = india_cases.sort_values('Confirmed', ascending=False)[:29]
horizontal_bar_chart(Confirmed_states_Cases,Confirmed_states_Cases.Confirmed, Confirmed_states_Cases.index,
            ' Indian States (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','States Name','Green')

In [ ]:
Recovered_states = india_cases.sort_values('Cured', ascending=False)[:29]
horizontal_bar_chart(Recovered_states, Recovered_states.Cured, Recovered_states.index,
                    ' States (Cured Cases)', 'Number of Cured cases (in Thousands)', 'States Name', 'Blue')

VACCINATION


In [ ]:
vaccination = pd.read_csv('/content/covid19/covid_vaccine_statewise.csv')
vaccination.tail()

In [ ]:
vaccination['Total Vaccinatons'] = vaccination['First Dose Administered']+vaccination['Second Dose Administered']

vaccination.rename(columns = {'Updated On':'Date'}, inplace = True)

In [ ]:
Maharashtra = vaccination[vaccination["State"]=="Maharashtra"]
fig = px.line(Maharashtra,x="Date",y="Total Vaccinatons",title="Vaccination till date in Maharashtra")  
fig.update_xaxes(rangeslider_visible=True)

In [ ]:
fig = px.line(Maharashtra,x="Date",y="Total CoviShield Administered",title="CoviShield Administered in Mahrashtra")  
fig.update_xaxes(rangeslider_visible=True) 

PROPHET FORCASTING


In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
from plotly.offline import iplot, init_notebook_mode

In [ ]:
model = Prophet()

In [ ]:
Confirmed = Trend.loc[:, ['Date', 'Confirmed']] 
Confirmed.tail()

In [ ]:
Cured = Trend.loc[:, ['Date', 'Cured']] 
Cured.tail()

In [ ]:
Deaths = Trend.loc[:, ['Date', 'Cured']] 
Deaths.tail()

In [ ]:
Confirmed.columns = ['ds', 'y']
model.fit(Confirmed)

In [ ]:
future = model.make_future_dataframe(periods=30) 
future.tail()

In [ ]:
forecast_india_conf = model.predict(future)
forecast_india_conf

In [ ]:
fig = plot_plotly(model, forecast_india_conf) 

fig.update_layout(template='plotly_white')

iplot(fig) 

In [ ]:
model1 = Prophet()

In [ ]:
Cured.columns = ['ds', 'y']
model1.fit(Cured)

future_1 = model1.make_future_dataframe(periods=30)
future_1.tail()

In [ ]:
forecast_india_cured = model1.predict(future_1)
forecast_india_cured

In [ ]:
fig = plot_plotly(model1, forecast_india_cured) 

fig.update_layout(template='plotly_white')

iplot(fig) 

In [ ]:
model2 = Prophet()

In [ ]:
Deaths.columns = ['ds', 'y']
model2.fit(Deaths)

future_2 = model2.make_future_dataframe(periods=30) 
future_2.tail()

In [ ]:
forecast_india_Deaths = model2.predict(future_2)
forecast_india_Deaths

In [ ]:
fig = plot_plotly(model1, forecast_india_cured) 

fig.update_layout(template='plotly_white')

iplot(fig) 